In [499]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [500]:
import re
import pandas as pd
from collections import defaultdict

def parse_pow_logs(file_path):
    mined_blocks = defaultdict(list)
    created_txs = defaultdict(list)

    with open(file_path, 'r') as f:
        for line in f:
            if "mined block" in line:
                m = re.search(
                    r"Node (\d+) mined block (\S+) at height (\d+) with (\d+) txs: \[(.*?)\]",
                    line
                )
                if m:
                    node, blk_hash, height, n_txs, txhash_list = m.groups()
                    txhashes = [tx.strip() for tx in txhash_list.split(",") if tx.strip()]
                    mined_blocks[node].append({
                        "block_hash": blk_hash,
                        "height": int(height),
                        "txs": int(n_txs),
                        "txhash_list": txhashes
                    })

            if "added new tx" in line:
                m = re.search(r"Node (\d+) added new tx (\S+) with fee (\d+) sat", line)
                if m:
                    node, txhash, fee = m.groups()
                    created_txs[node].append({
                        "txhash": txhash,
                        "fee": int(fee)
                    })

    mined_df = pd.DataFrame(
        [ {"node": node, **blk} 
          for node, blks in mined_blocks.items() 
          for blk in blks ]
    )
    tx_df = pd.DataFrame(
        [ {"node": node, **tx} 
          for node, txs in created_txs.items() 
          for tx in txs ]
    )

    return mined_df, tx_df


In [501]:
log_path = "src/ledger_logs.txt"

mined_df, tx_df = parse_pow_logs(log_path)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(mined_df)


,node,block_hash,height,txs,txhash_list
0,7,-1495378322769522153,0,8,"[6e6c023e, e0944cb7, 1f82d6f9, 9cc43bba, a28db..."
1,7,-1495378322769522153,0,8,"[6e6c023e, e0944cb7, 1f82d6f9, 9cc43bba, a28db..."
2,7,-7575335743330580815,4,26,"[40087847, aea43da8, f55b4a02, e2cf3d7d, 9f776..."
3,7,-7632263321476713586,5,6,"[a684bfd4, eb6470ba, f8b949d3, 47b291f8, 3e761..."
4,7,3843119688891745575,18,42,"[7cbaca9c, 507743df, 34c5e0f, 424c9f8c, a94a23..."
5,7,4085721342808365650,25,16,"[b4595dbf, bf20c22, f5c8e5b8, fc2f3a0f, 6db00c..."
6,7,6417943338423207005,45,82,"[56cb575, 8bb61e5c, b6d3346a, de3d156b, b8ce86..."
7,7,9604252739661002,51,36,"[abd4377b, 9357fd31, ffe98385, d6dd0011, 95cd0..."
8,7,-1213322772515475998,52,3,"[cd6f285, 21cc18be, ddb54c95]"
9,7,-6824645182669753745,64,71,"[7b5297e5, 6073be51, d2c289b3, 253856be, 88164..."


In [502]:
groups = defaultdict(list)
for node, chain in node_chains.items():
    groups[tuple(chain)].append(node)

# pick the chain held by the most nodes
ref_chain, ref_nodes = max(groups.items(), key=lambda kv: len(kv[1]))

ref_chain = list(ref_chain)


In [503]:
# 1) Filter mined_df to only the blocks on the reference chain
main_chain_df = (
    mined_df
      .drop_duplicates('block_hash')
      .loc[lambda df: df['block_hash'].isin(ref_chain)]
)

# 2) Reorder rows to match the reference‐chain order
main_chain_df = (
    main_chain_df
      .set_index('block_hash')
      .loc[ref_chain]
      .reset_index()
)

# 3) Display the main chain with its miner, height, and included txs
display(main_chain_df[['node', 'block_hash', 'height', 'txs', 'txhash_list']])


,node,block_hash,height,txs,txhash_list
0,7,-1495378322769522153,0,8,"[6e6c023e, e0944cb7, 1f82d6f9, 9cc43bba, a28db..."
1,4,-5321073299589362089,1,14,"[83355d0d, 260c504, 3d4f4838, c6ca9b44, a6dd44..."
2,6,-8310903792393162389,2,29,"[957df5dc, 859f469c, c319b8e6, 739ea2a4, a4f89..."
3,2,1532840136865757527,3,24,"[d76e4ed7, 4fc8359e, c7aaa60a, 4f8a7444, 97efa..."
4,7,-7575335743330580815,4,26,"[40087847, aea43da8, f55b4a02, e2cf3d7d, 9f776..."
5,7,-7632263321476713586,5,6,"[a684bfd4, eb6470ba, f8b949d3, 47b291f8, 3e761..."
6,3,7976257456511193773,6,49,"[db410195, 4b49a83d, 8d64723e, 1f7f2fa, 37666a..."
7,4,-2560725219497095111,7,32,"[f746d0b8, 43bd426d, dc228f06, cea1af7d, 4c521..."
8,1,-6209479357270152831,8,39,"[42c7503f, 239cf8c5, ac24bac7, c7feda53, 576eb..."
9,2,-5410179612771026693,9,22,"[265f0686, d30970d, 3e874beb, 7e660296, 2c617a..."


In [504]:
display(tx_df)

,node,txhash,fee
0,5,4a1b9d,178
1,5,a1c346,116
2,5,381953,10
3,5,fb6735,176
4,5,e16765,14
5,5,5bf87d,38
6,5,531ef4,5
7,5,d566c5,31
8,5,4d31a8,12
9,5,e714c9,6


In [505]:
# 1) Blocks mined per node (unchanged)
print("\nBlocks Mined per Node")
blocks_mined = mined_df.groupby("node")["block_hash"] \
                       .count() \
                       .rename("blocks_mined")
print(blocks_mined)

# 2) Transactions mined per node, from actual txhash_list
print("\nTransactions Mined per Node")
txs_mined = mined_df.groupby("node")["txhash_list"] \
    .apply(lambda lists: sum(len(lst) for lst in lists)) \
    .rename("txs_mined")
print(txs_mined)

print(f"\nTotal transactions mined across network: {txs_mined.sum()}")

# 3) Transactions created per node (unique txhashes)
print("\nTransactions Created per Node")
txs_created = tx_df.groupby("node")["txhash"] \
                   .nunique() \
                   .rename("txs_created")
print(txs_created)

print(f"\nTotal unique transactions created: {txs_created.sum()}")



Blocks Mined per Node
node
0     6
1    12
2    16
3     8
4    13
5    12
6    10
7    12
8     9
9    10
Name: blocks_mined, dtype: int64

Transactions Mined per Node
node
0    345
1    335
2    364
3    359
4    381
5    395
6    383
7    365
8    338
9    289
Name: txs_mined, dtype: int64

Total transactions mined across network: 3554

Transactions Created per Node
node
0    427
1    350
2    380
3    406
4    383
5    398
6    408
7    396
8    365
9    376
Name: txs_created, dtype: int64

Total unique transactions created: 3889


In [506]:
print("Total number of Blocks mined per Node")
print(mined_df['node'].value_counts().rename("blocks_mined"))

Total number of Blocks mined per Node
node
2    16
4    13
7    12
1    12
5    12
6    10
9    10
8     9
3     8
0     6
Name: blocks_mined, dtype: int64


In [507]:
all_created_txids = set(tx_df['txhash'])
print(f"\nTotal unique transactions created: {len(all_created_txids)}")


Total unique transactions created: 3887


In [508]:
total_mined_txs = mined_df['txhash_list'].apply(len).sum()
print(f"Total transactions mined: {total_mined_txs}")

Total transactions mined: 3554


In [509]:
left_in_mempool = len(all_created_txids) - total_mined_txs
print(f"Unique txs left in mempools: {left_in_mempool}")

Unique txs left in mempools: 333


In [510]:
# 1) Restrict to canonical blocks only
ref_mined_df = mined_df[mined_df['block_hash'].isin(ref_chain)] \
                  .drop_duplicates('block_hash')

# 2) Total slots on the final chain (every tx inclusion)
total_slots_final = ref_mined_df['txhash_list'].apply(len).sum()

# 3) Unique tx‐hashes on the final chain
all_final_txhashes = {
    tx for lst in ref_mined_df['txhash_list'] 
       for tx in lst
}
total_unique_final = len(all_final_txhashes)

print(f"Total slots in final chain:       {total_slots_final}")
print(f"Total unique txs in final chain:  {total_unique_final}")


Total slots in final chain:       3354
Total unique txs in final chain:  3354


In [511]:
# For each node, collect the set of tx-hashes it contributed to canonical blocks
unique_per_node = (
    ref_mined_df
      .groupby('node')['txhash_list']
      .apply(lambda lists: {tx for lst in lists for tx in lst})
      .rename('unique_txset')
)

# Convert to counts
unique_counts = unique_per_node.apply(len).rename('unique_txs_in_final_chain')

print("Per-node unique transactions in final chain:")
print(unique_counts)


Per-node unique transactions in final chain:
node
0    345
1    310
2    364
3    345
4    381
5    395
6    279
7    357
8    300
9    278
Name: unique_txs_in_final_chain, dtype: int64


In [512]:
# 1) All created tx hashes
all_created = set(tx_df['txhash'])
# 2) All tx hashes that survived in the canonical chain
all_final   = set(
    tx for lst in ref_mined_df['txhash_list'] 
        for tx in lst
)

never_mined = all_created - all_final

print(f"Total unique TXs created:           {len(all_created)}")
print(f"Total unique TXs in final chain:    {len(all_final)}")
print(f"TXs never mined (leftover):         {len(never_mined)} "
      f"({len(never_mined)/len(all_created):.2%})")


Total unique TXs created:           3887
Total unique TXs in final chain:    3354
TXs never mined (leftover):         3870 (99.56%)


How many txs are mined into the final canonical chain?

In [513]:
# 1) Build a set of blocks in the reference chain
ref_blocks = set(ref_chain)

# 2) Filter to only those mined‐block rows whose block_hash is in the ref chain
ref_mined_df = mined_df[mined_df['block_hash'].isin(ref_blocks)]

# 3) Group by node and sum the lengths of each block’s txhash_list
txs_in_final = (
    ref_mined_df
      .groupby('node')['txhash_list']
      .apply(lambda lists: sum(len(lst) for lst in lists))
      .rename('txs_in_final_chain')
)

print("Transactions mined into the final chain per node:")
print(txs_in_final)


Transactions mined into the final chain per node:
node
0    345
1    310
2    364
3    345
4    381
5    395
6    279
7    365
8    300
9    278
Name: txs_in_final_chain, dtype: int64


In [514]:
total_txs_in_chain = ref_mined_df['txhash_list'].apply(len).sum()

print(f"Total transactions mined into final canonical chain: {total_txs_in_chain}")


Total transactions mined into final canonical chain: 3362


# EVALUATE CHAINS FOR EACH NODE TO GET CONSENSUS METRICS

In [515]:
with open("src/ledger_logs.txt") as f:
    text = f.read()

node_ids = sorted(set(re.findall(r"Node (\d+)", text)))
print("All node IDs seen in log:", node_ids)

All node IDs seen in log: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [516]:
for node in node_ids:
    df_node = mined_df[mined_df['node'] == str(node)]
    blocks = len(df_node)

    total_txs = df_node['txhash_list'].apply(len).sum()
    print(f"Node {node}: mined {blocks:3d} blocks, total txs = {total_txs:3d}")


Node 0: mined   6 blocks, total txs = 345
Node 1: mined  12 blocks, total txs = 335
Node 2: mined  16 blocks, total txs = 364
Node 3: mined   8 blocks, total txs = 359
Node 4: mined  13 blocks, total txs = 381
Node 5: mined  12 blocks, total txs = 395
Node 6: mined  10 blocks, total txs = 383
Node 7: mined  12 blocks, total txs = 365
Node 8: mined   9 blocks, total txs = 338
Node 9: mined  10 blocks, total txs = 289


In [517]:
import re
from collections import defaultdict

log_path = "src/ledger_logs.txt"

events = []
pattern_genesis = re.compile(r"Node (\d+): genesis block (\S+) added")
pattern_add     = re.compile(r"Node (\d+): added block (\S+) under parent (\S+)")

with open(log_path) as f:
    for line in f:
        m = pattern_genesis.search(line)
        if m:
            node, blk = m.groups()
            events.append((int(node), blk, None))
            continue
        m = pattern_add.search(line)
        if m:
            node, blk, parent = m.groups()
            events.append((int(node), blk, parent))

# 2) Build parent maps per node
parent_map = defaultdict(dict)   # node_id → { block_hash: prev_hash }
for node, blk, parent in events:
    parent_map[node][blk] = parent

# 3) For each node, find its longest chain by walking from each leaf back to genesis
def longest_chain_for(node_id):
    pm = parent_map[node_id]
    all_blocks = set(pm.keys())
    parents   = {p for p in pm.values() if p is not None}
    leaves    = all_blocks - parents
    best_chain = []
    for leaf in leaves:
        chain = []
        cur = leaf
        while cur is not None:
            chain.append(cur)
            cur = pm.get(cur, None)
        chain.reverse()  # genesis → leaf
        if len(chain) > len(best_chain):
            best_chain = chain
    return best_chain

# Reconstruct every node’s best chain
node_chains = {
    node: longest_chain_for(node)
    for node in sorted(parent_map)
}

for node, chain in node_chains.items():
    print(f"Node {node:2d}: chain length = {len(chain)}  tip = {chain[-1]}")


Node  0: chain length = 101  tip = 7614014854439318064
Node  1: chain length = 101  tip = 7614014854439318064
Node  2: chain length = 101  tip = 7614014854439318064
Node  3: chain length = 101  tip = 7614014854439318064
Node  4: chain length = 101  tip = 7614014854439318064
Node  5: chain length = 101  tip = 7614014854439318064
Node  6: chain length = 101  tip = 7614014854439318064
Node  7: chain length = 101  tip = 7614014854439318064
Node  8: chain length = 101  tip = 7614014854439318064
Node  9: chain length = 101  tip = 7614014854439318064


In [518]:
from collections import defaultdict

groups = defaultdict(list) # Group nodes by identical chains
for node, chain in node_chains.items():
    groups[tuple(chain)].append(node)

print(f"Found {len(groups)} distinct chain(s) among {len(node_chains)} nodes.\n")
for chain, members in groups.items():
    print(f" • Chain of length {len(chain)} → nodes {members}")

ref_chain, ref_nodes = max(groups.items(), key=lambda kv: len(kv[1]))
print(f"\nReference chain (held by {len(ref_nodes)} nodes): length={len(ref_chain)}") # Chain that is shared by majority is the canonical chain used as a reference

def common_prefix_len(a, b): # get length of how many blocks in common with canonical
    for i, (x, y) in enumerate(zip(a, b)):
        if x != y:
            return i
    return min(len(a), len(b))

print("\nCommon‐prefix lengths:")
for node, chain in node_chains.items():
    cpl = common_prefix_len(ref_chain, chain)
    print(f" Node {node:2d}: {cpl} blocks in common with reference")


Found 1 distinct chain(s) among 10 nodes.

 • Chain of length 101 → nodes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Reference chain (held by 10 nodes): length=101

Common‐prefix lengths:
 Node  0: 101 blocks in common with reference
 Node  1: 101 blocks in common with reference
 Node  2: 101 blocks in common with reference
 Node  3: 101 blocks in common with reference
 Node  4: 101 blocks in common with reference
 Node  5: 101 blocks in common with reference
 Node  6: 101 blocks in common with reference
 Node  7: 101 blocks in common with reference
 Node  8: 101 blocks in common with reference
 Node  9: 101 blocks in common with reference


# Orphan Rate
#### Count total “mined block” events vs. blocks that appear in each node’s final canonical-hain.

#### A lower orphan rate (→ 5–15% in real Bitcoin) means good propagation

#### This is calculated as orphan_rate = (no. of blocks mined by the node but NOT in its current best chain) / (total blocks the node mined)


In [519]:

# 1) Total unique blocks mined across all nodes
all_mined_hashes = set(mined_df['block_hash'])
total_mined = len(all_mined_hashes)
print(f"Total unique blocks mined across network: {total_mined}")

# 2) Network‐level orphan rate using the reference chain
blocks_in_ref = len(ref_chain)
network_orphan_rate = 1 - blocks_in_ref / total_mined
print(f"Blocks in reference chain:        {blocks_in_ref}")
print(f"Network orphan rate:             {network_orphan_rate:.2%}")

# 3) Per‐node orphan rates
global_ref = set(ref_chain)
print("\nCorrected per-node orphan rates:")
print("\nCorrected per-node orphan rates:")
for node in node_ids:
    mined = set(mined_df.loc[mined_df['node'] == str(node), 'block_hash'])
    adopted = len(mined & global_ref)
    orphaned = len(mined) - adopted
    rate = orphaned / len(mined) if mined else 0
    print(f"Node {node}: mined {len(mined)}, orphaned {orphaned}, adopted {adopted}, orphan rate {rate:.2%}")

Total unique blocks mined across network: 107
Blocks in reference chain:        101
Network orphan rate:             5.61%

Corrected per-node orphan rates:

Corrected per-node orphan rates:
Node 0: mined 6, orphaned 0, adopted 6, orphan rate 0.00%
Node 1: mined 12, orphaned 1, adopted 11, orphan rate 8.33%
Node 2: mined 16, orphaned 0, adopted 16, orphan rate 0.00%
Node 3: mined 8, orphaned 1, adopted 7, orphan rate 12.50%
Node 4: mined 13, orphaned 0, adopted 13, orphan rate 0.00%
Node 5: mined 12, orphaned 0, adopted 12, orphan rate 0.00%
Node 6: mined 10, orphaned 2, adopted 8, orphan rate 20.00%
Node 7: mined 11, orphaned 0, adopted 11, orphan rate 0.00%
Node 8: mined 9, orphaned 1, adopted 8, orphan rate 11.11%
Node 9: mined 10, orphaned 1, adopted 9, orphan rate 10.00%


# Transaction Confirmation Rates

#### How many transactions created actually made it into the final best‐chain?

#### For each tx: find the first block in the reference chain containing it, then measure its depth (e.g. ≥ 6 blocks) to see how “final” it became.

In [520]:
import pandas as pd

# 1) Build ref_mined_df exactly as before
ref_mined_df = mined_df[mined_df['block_hash'].isin(ref_chain)] \
                  .drop_duplicates('block_hash')

# 2) Map each ref-chain block to its real tx list
block_tx_map = dict(zip(ref_mined_df['block_hash'],
                        ref_mined_df['txhash_list']))

# 3) Walk the reference chain and record first‐index per tx
tx_first_idx = {}
for idx, blk in enumerate(ref_chain):
    for tx in block_tx_map.get(blk, []):
        tx_first_idx.setdefault(tx, idx)

# 4) Build the set of all created tx‐hashes
all_created = set(tx_df['txhash'])

records = []
chain_len = len(ref_chain)
for tx in all_created:
    first = tx_first_idx.get(tx)          # None if never included
    depth = (chain_len - first) if first is not None else 0
    records.append({
        'txhash':      tx,
        'first_index': first,
        'depth':        depth,
        'confirmed_6':  depth >= 6
    })

conf_df = pd.DataFrame(records)

# 5) Summary
total    = len(conf_df)
included = conf_df['first_index'].notnull().sum()
c6       = conf_df['confirmed_6'].sum()

print(f"Total TXs created:                {total}")
print(f"TXs in final chain:               {included} ({included/total:.2%})")
print(f"TXs ≥6 confirmations:             {c6} ({c6/total:.2%})\n")

display(conf_df.sort_values(by='depth', ascending=False))

Total TXs created:                3887
TXs in final chain:               17 (0.44%)
TXs ≥6 confirmations:             16 (0.41%)


,txhash,first_index,depth,confirmed_6
788,a23b2e,12.0,89,True
3057,9388e4,13.0,88,True
611,2e5598,23.0,78,True
3507,fc9bd9,28.0,73,True
864,664f1f,44.0,57,True
73,cd3bcd,45.0,56,True
3737,95195e,48.0,53,True
2262,b5d8ae,53.0,48,True
3038,47552c,53.0,48,True
3299,939328,55.0,46,True


# Throughput & Utilization

#### Average txs per block.

#### % of mempool emptied by mining vs. left over (txs created vs. txs mined).

In [521]:
import re

log_file = 'src/ledger_logs.txt'

created_txs     = []
txs_per_block   = []

with open(log_file) as f:
    for line in f:
        if 'added new tx' in line:
            m = re.search(r'new tx (\S+)', line)
            if m:
                created_txs.append(m.group(1))

        if 'mined block' in line and 'with' in line and 'txs' in line:
            m = re.search(r'with (\d+) txs', line)
            if m:
                txs_per_block.append(int(m.group(1)))

unique_created = set(created_txs)
total_created  = len(unique_created)
total_mined    = sum(txs_per_block)
num_blocks     = len(txs_per_block)

avg_txs_per_blk = total_mined / num_blocks if num_blocks else 0
pct_mined       = total_mined / total_created if total_created else 0
pct_leftover    = 1 - pct_mined

print(f"Blocks mined (log):           {num_blocks}")
print(f"Average txs per block:        {avg_txs_per_blk:.2f}")
print(f"Total unique TXs created:     {total_created}")
print(f"Total TXs mined (events):     {total_mined}")
print(f"% TXs mined (utilization):    {pct_mined:.2%}")
print(f"% TXs left in mempool/log:    {pct_leftover:.2%}")


Blocks mined (log):           108
Average txs per block:        32.91
Total unique TXs created:     3887
Total TXs mined (events):     3554
% TXs mined (utilization):    91.43%
% TXs left in mempool/log:    8.57%


In [522]:
import re
from collections import defaultdict

log_file = 'src/ledger_logs.txt'

created_txs = defaultdict(list)
mined_txs_per_node = defaultdict(int)
blocks_mined = defaultdict(int)

with open(log_file, 'r') as f:
    for line in f:
        m = re.search(r'Node (\S+) added new tx (\S+) with fee', line)
        if m:
            node, txhash = m.group(1), m.group(2)
            created_txs[node].append(txhash)
            continue

        m = re.search(
            r'Node (\S+) mined block \S+ at height \d+ with \d+ txs: \[(.*?)\]',
            line
        )
        if m:
            node, txhash_list = m.group(1), m.group(2)
            # split on commas, strip whitespace
            txs = [tx.strip() for tx in txhash_list.split(',') if tx.strip()]
            mined_txs_per_node[node] += len(txs)
            blocks_mined[node]       += 1

print("Per-node Throughput & Utilization:\n")
all_nodes = sorted(set(created_txs) | set(blocks_mined), key=lambda x: int(x))
for node in all_nodes:
    created = len(created_txs.get(node, []))
    mined   = mined_txs_per_node.get(node, 0)
    blocks  = blocks_mined.get(node, 0)

    avg_txs_per_block = mined / blocks if blocks else 0
    pct_mined         = mined / created if created else 0
    pct_left          = 1 - pct_mined if created else 0

    print(f"Node {node}:")
    print(f"  TXs created:       {created}")
    print(f"  Blocks mined:      {blocks}")
    print(f"  Avg txs per block: {avg_txs_per_block:.2f}")
    print(f"  TXs mined:         {mined} ({pct_mined:.2%})")
    print(f"  TXs left over:     {created - mined} ({pct_left:.2%})\n")


Per-node Throughput & Utilization:

Node 0:
  TXs created:       427
  Blocks mined:      6
  Avg txs per block: 57.50
  TXs mined:         345 (80.80%)
  TXs left over:     82 (19.20%)

Node 1:
  TXs created:       350
  Blocks mined:      12
  Avg txs per block: 27.92
  TXs mined:         335 (95.71%)
  TXs left over:     15 (4.29%)

Node 2:
  TXs created:       380
  Blocks mined:      16
  Avg txs per block: 22.75
  TXs mined:         364 (95.79%)
  TXs left over:     16 (4.21%)

Node 3:
  TXs created:       406
  Blocks mined:      8
  Avg txs per block: 44.88
  TXs mined:         359 (88.42%)
  TXs left over:     47 (11.58%)

Node 4:
  TXs created:       383
  Blocks mined:      13
  Avg txs per block: 29.31
  TXs mined:         381 (99.48%)
  TXs left over:     2 (0.52%)

Node 5:
  TXs created:       398
  Blocks mined:      12
  Avg txs per block: 32.92
  TXs mined:         395 (99.25%)
  TXs left over:     3 (0.75%)

Node 6:
  TXs created:       408
  Blocks mined:      10
  A

# VERIFY THAT MATCHING CHAINS HAVE THE SAME BLOCKS MINED

In [523]:
inconsistencies = []
for block_hash, grp in mined_df.groupby("block_hash"):
    lists = grp["txhash_list"].tolist()

    if len(lists) > 1 and any(lists[0] != other for other in lists[1:]):
        inconsistencies.append({
            "block_hash": block_hash,
            "variants": lists
        })

if inconsistencies:
    print("FOUND inconsistent tx lists for these blocks:")
    for bad in inconsistencies:
        print(" ", bad["block_hash"], bad["variants"])
else:
    print("All blocks with the same hash have identical tx lists.")


All blocks with the same hash have identical tx lists.


# Measure Branching Ratio

### Branching ratio is the avg number of child blocks makes, including the canonical one
### if chain is linear and has no forks, the branching ratio will be 1 as only ever one child block

#### Branching ratio = 1 + (B-C/C)
#### Where B is the total number of blocks in your dataset (including orphans) and C is total number of canonical blocks (blocks in your reference chain)

In [524]:
from collections import Counter

parent_counts = Counter(mined_df['prev_hash'].dropna())
avg_children = sum(parent_counts.values()) / len(parent_counts)
print(f"Branching ratio (children per block): {avg_children:.2f}")


KeyError: 'prev_hash'

In [ ]:
# 7.2 TXs in final chain
final_txhashes = {tx for lst in main_chain_df['txhash_list'] for tx in lst}
total_final = len(final_txhashes)
print(f"TXs in final chain: {total_final}")

In [ ]:
# Transactions with ≥6 confirmations
# Build first-index map from main chain data
chain_blocks = main_chain_df.set_index('block_hash').loc[ref_chain]
tx_first_idx = {}
for idx, (blk_hash, row) in enumerate(chain_blocks.iterrows()):
    for tx in row['txhash_list']:
        tx_first_idx.setdefault(tx, idx)
# Compute depths and count those ≥6 confirmations
depths = [len(ref_chain) - tx_first_idx.get(tx, 0) for tx in final_txhashes]
total_ge6 = sum(1 for d in depths if d >= 6)
print(f"TXs ≥6 confirmations: {total_ge6}")